In [1]:
library(scMultiSim)
library(dplyr)
library(reticulate)  # For Python integration to save .h5ad

# Load Python's anndata package (must be installed in your Python env)
anndata <- import("anndata")

# Load GRN parameters
data(GRN_params_100, envir = environment())

# # Define ligand parameters
lig_params <- data.frame(
  target    = c(101, 102),
  regulator = c(103, 104),
  effect    = c(5.2, 5.9)
)

# Define spatial options function
spatial_options <- function(cell_count, layout_type, ...) {
  cci_opt <- list(
    params = lig_params,
    max.neighbors = 6,
    start.layer = cell_count,
    grid.size = 128,
    cell.type.interaction = "random"
  )
  list(
    rand.seed = 0,
    GRN = GRN_params_100,
    num.cells = cell_count,
    num.cifs = 60,
    num.genes = 200,
    cif.sigma = 0.6,
    tree = Phyla5(),
    diff.cif.fraction = 0.9,
    speed.up = TRUE,
    cci = c(cci_opt, list(layout = layout_type))
  )
}

# Define cell counts and layouts to test
cell_counts <- c(500, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000)
layouts <- c("enhanced")

# Create all combinations (10 total)
sim_params <- expand.grid(
  cell_count = cell_counts,
  layout_type = layouts
)

save_results <- function(results, dir_path, sim_name) {
  # Create directory if it doesn't exist
  if (!dir.exists(dir_path)) {
    dir.create(dir_path, recursive = TRUE)
  }
  
  # Save data as CSV (optional)
  write.csv(results$counts, file = file.path(dir_path, "rna_counts.csv"), row.names = TRUE)
  write.csv(results$atac_counts, file = file.path(dir_path, "atac_counts.csv"), row.names = TRUE)
  write.csv(results$cci_locs, file = file.path(dir_path, "cci_locs.csv"), row.names = TRUE)
  write.csv(results$cell_meta, file = file.path(dir_path, "cell_meta.csv"), row.names = TRUE)
  
  # Save plots as PDF (higher quality)
  pdf(file.path(dir_path, "tsne_plot.pdf"), width = 8, height = 6)
  plot_tsne(log2(results$counts + 1),
           results$cell_meta$pop,
           legend = 'pop', plot.name = paste('True RNA Counts Tsne -', sim_name))
  dev.off()
  
  pdf(file.path(dir_path, "correlation_heatmap.pdf"), width = 8, height = 6)
  plot_gene_module_cor_heatmap(results)
  dev.off()

  pdf(file.path(dir_path, "spatial_distribution.pdf"), width = 8, height = 6)
  plot_cell_loc(results, show.arrows = FALSE)
  dev.off()
  
  # Convert cci_locs to matrix format for obsm
  spatial_coords <- as.matrix(results$cci_locs)
  rownames(spatial_coords) <- rownames(results$cell_meta)
  
  # Save as H5AD (Python anndata format)
  adata_rna <- anndata$AnnData(
    X = t(results$counts),  # Cells × Genes
    obs = results$cell_meta,
    obsm = list(spatial = spatial_coords)  # Add spatial coordinates to obsm
  )
  adata_rna$write(file.path(dir_path, "sim_RNA.h5ad"))
  
  if (!is.null(results$atac_counts)) {
    adata_atac <- anndata$AnnData(
      X = t(results$atacseq_data),
      obs = results$cell_meta,
      obsm = list(spatial = spatial_coords)  # Same spatial coordinates for ATAC
    )
    adata_atac$write(file.path(dir_path, "sim_ATAC.h5ad"))
  }
}

# Generate all simulations
for (i in 1:nrow(sim_params)) {
  cell_count <- sim_params$cell_count[i]
  layout_type <- sim_params$layout_type[i]
  
  # Create simulation name and directory path
  sim_name <- paste0("sim_", i)
  dir_path <- file.path("scMultiSim_data", sim_name)

  if (!dir.exists(dir_path)) {
    dir.create(dir_path, recursive = TRUE)
    cat("Created directory:", dir_path, "\n")
  }
  
  cat("Generating simulation:", sim_name, "\n")
  
  # Run simulation
  results <- sim_true_counts(spatial_options(cell_count, layout_type))
  add_expr_noise(
    results,
    # options go here
    alpha_mean = 1e4
  )
  divide_batches(
    results,
    nbatch = 2,
    effect = 1
  )
  # Save results
  save_results(results, dir_path, sim_name)
}

cat("All simulations completed!\n")


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




Generating simulation: sim_1 


Experimental speed optimization enabled.



CCI simulation is enabled.


Spatial: only the last layer will be simulated.



...

CCI simulation is enabled.


Spatial: only the last layer will be simulated.



Get CIF...100..200..300..400..500..600..700..800..900..1000..1100..1200..1300..1400..1500..1600..1700..1800..1900..2000..2100..2200..2300..2400..2500..2600..2700..2800..2900..3000..3100..3200..3300..3400..3500..3600..3700..3800..3900..4000..4100..4200..4300..4400..4500..4600..4700..4800..4900..5000..Done
Get params...Done
Simulating...5000..
Time spent: 13.85 mins
Adding experimental noise...
50..100..150..200..250..300..350..400..450..500..550..600..650..700..750..800..850..900..950..1000..1050..1100..1150..1200..1250..1300..1350..1400..1450..1500..1550..1600..1650..1700..1750..1800..1850..1900..1950..2000..2050..2100..2150..2200..2250..2300..2350..2400..2450..2500..2550..2600..2650..2700..2750..2800..2850..2900..2950..3000..3050..3100..3150..3200..3250..3300..3350..3400..3450..3500..3550..3600..3650..3700..3750..3800..3850..3900..3950..4000..4050..4100..4150..4200..4250..4300..4350..4400..4450..4500..4550..4600..4650..4700..4750..4800..4850..4900..4950..5000..Using atac_counts


Time spent: 9.95 mins




Adding batch effects...
All simulations completed!
